In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits += tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/3, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/3, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:14.0000 R:14.0000 rate:0.0280 aloss:0.6939 eloss:2.0795 aloss2:1.3795 exploreP:0.9986
Episode:1 meanR:20.0000 R:26.0000 rate:0.0520 aloss:0.6956 eloss:2.0554 aloss2:1.3984 exploreP:0.9960
Episode:2 meanR:26.3333 R:39.0000 rate:0.0780 aloss:0.6893 eloss:2.0542 aloss2:1.3916 exploreP:0.9922
Episode:3 meanR:24.5000 R:19.0000 rate:0.0380 aloss:0.6913 eloss:2.0389 aloss2:1.4002 exploreP:0.9903
Episode:4 meanR:27.4000 R:39.0000 rate:0.0780 aloss:0.6859 eloss:2.0404 aloss2:1.3942 exploreP:0.9865
Episode:5 meanR:24.6667 R:11.0000 rate:0.0220 aloss:0.6852 eloss:2.0343 aloss2:1.3930 exploreP:0.9855
Episode:6 meanR:22.7143 R:11.0000 rate:0.0220 aloss:0.6905 eloss:2.0240 aloss2:1.3969 exploreP:0.9844
Episode:7 meanR:23.2500 R:27.0000 rate:0.0540 aloss:0.6874 eloss:2.0335 aloss2:1.3960 exploreP:0.9818
Episode:8 meanR:22.6667 R:18.0000 rate:0.0360 aloss:0.6845 eloss:2.0151 aloss2:1.4023 exploreP:0.9800
Episode:9 meanR:24.7000 R:43.0000 rate:0.0860 aloss:0.6871 eloss:2.0216 aloss2:1.3

Episode:80 meanR:25.0617 R:28.0000 rate:0.0560 aloss:0.6962 eloss:1.5881 aloss2:1.9556 exploreP:0.8181
Episode:81 meanR:24.9512 R:16.0000 rate:0.0320 aloss:0.6845 eloss:1.5614 aloss2:2.0249 exploreP:0.8168
Episode:82 meanR:25.2651 R:51.0000 rate:0.1020 aloss:0.6828 eloss:1.5635 aloss2:1.9493 exploreP:0.8127
Episode:83 meanR:25.3571 R:33.0000 rate:0.0660 aloss:0.6879 eloss:1.5554 aloss2:2.0140 exploreP:0.8101
Episode:84 meanR:25.1647 R:9.0000 rate:0.0180 aloss:0.6761 eloss:1.5409 aloss2:2.0414 exploreP:0.8094
Episode:85 meanR:25.6744 R:69.0000 rate:0.1380 aloss:0.6892 eloss:1.5521 aloss2:1.9832 exploreP:0.8039
Episode:86 meanR:25.5172 R:12.0000 rate:0.0240 aloss:0.6916 eloss:1.5600 aloss2:1.9667 exploreP:0.8029
Episode:87 meanR:25.7841 R:49.0000 rate:0.0980 aloss:0.6928 eloss:1.5447 aloss2:2.0329 exploreP:0.7990
Episode:88 meanR:25.7416 R:22.0000 rate:0.0440 aloss:0.6996 eloss:1.5459 aloss2:2.0863 exploreP:0.7973
Episode:89 meanR:25.6556 R:18.0000 rate:0.0360 aloss:0.6872 eloss:1.5415 a

Episode:159 meanR:29.8400 R:43.0000 rate:0.0860 aloss:0.6830 eloss:1.3145 aloss2:2.7954 exploreP:0.6447
Episode:160 meanR:29.9800 R:33.0000 rate:0.0660 aloss:0.6894 eloss:1.3203 aloss2:2.7492 exploreP:0.6426
Episode:161 meanR:30.1800 R:33.0000 rate:0.0660 aloss:0.6889 eloss:1.3340 aloss2:2.8496 exploreP:0.6406
Episode:162 meanR:30.2500 R:27.0000 rate:0.0540 aloss:0.6898 eloss:1.3175 aloss2:2.7854 exploreP:0.6389
Episode:163 meanR:30.7500 R:69.0000 rate:0.1380 aloss:0.6960 eloss:1.3077 aloss2:2.8068 exploreP:0.6345
Episode:164 meanR:30.5800 R:28.0000 rate:0.0560 aloss:0.6958 eloss:1.3345 aloss2:2.8836 exploreP:0.6328
Episode:165 meanR:30.3800 R:23.0000 rate:0.0460 aloss:0.7011 eloss:1.3205 aloss2:2.8126 exploreP:0.6314
Episode:166 meanR:30.5100 R:31.0000 rate:0.0620 aloss:0.7033 eloss:1.2897 aloss2:2.8625 exploreP:0.6294
Episode:167 meanR:30.8300 R:52.0000 rate:0.1040 aloss:0.7112 eloss:1.3053 aloss2:2.8264 exploreP:0.6262
Episode:168 meanR:30.9300 R:31.0000 rate:0.0620 aloss:0.6912 elo

Episode:238 meanR:39.0600 R:54.0000 rate:0.1080 aloss:0.6839 eloss:1.2372 aloss2:3.1818 exploreP:0.4795
Episode:239 meanR:39.1400 R:35.0000 rate:0.0700 aloss:0.6828 eloss:1.2283 aloss2:3.1937 exploreP:0.4778
Episode:240 meanR:39.2800 R:30.0000 rate:0.0600 aloss:0.6824 eloss:1.2131 aloss2:3.1758 exploreP:0.4764
Episode:241 meanR:39.2200 R:32.0000 rate:0.0640 aloss:0.6812 eloss:1.2268 aloss2:3.2132 exploreP:0.4749
Episode:242 meanR:39.2600 R:42.0000 rate:0.0840 aloss:0.6817 eloss:1.2111 aloss2:3.2150 exploreP:0.4730
Episode:243 meanR:39.3700 R:46.0000 rate:0.0920 aloss:0.6806 eloss:1.2360 aloss2:3.2324 exploreP:0.4709
Episode:244 meanR:39.6100 R:41.0000 rate:0.0820 aloss:0.6851 eloss:1.1955 aloss2:3.2237 exploreP:0.4690
Episode:245 meanR:39.3500 R:28.0000 rate:0.0560 aloss:0.6801 eloss:1.2146 aloss2:3.2309 exploreP:0.4677
Episode:246 meanR:38.8100 R:46.0000 rate:0.0920 aloss:0.6797 eloss:1.2016 aloss2:3.2386 exploreP:0.4656
Episode:247 meanR:39.0800 R:54.0000 rate:0.1080 aloss:0.6851 elo

Episode:317 meanR:37.3900 R:30.0000 rate:0.0600 aloss:0.7306 eloss:1.2056 aloss2:3.2612 exploreP:0.3636
Episode:318 meanR:37.5300 R:41.0000 rate:0.0820 aloss:0.7155 eloss:1.1924 aloss2:3.2657 exploreP:0.3621
Episode:319 meanR:37.9500 R:62.0000 rate:0.1240 aloss:0.7422 eloss:1.2024 aloss2:3.2506 exploreP:0.3600
Episode:320 meanR:37.9400 R:44.0000 rate:0.0880 aloss:0.7105 eloss:1.1892 aloss2:3.2762 exploreP:0.3584
Episode:321 meanR:37.9300 R:30.0000 rate:0.0600 aloss:0.7303 eloss:1.1932 aloss2:3.2947 exploreP:0.3574
Episode:322 meanR:38.1100 R:43.0000 rate:0.0860 aloss:0.7138 eloss:1.2109 aloss2:3.2849 exploreP:0.3559
Episode:323 meanR:38.0200 R:29.0000 rate:0.0580 aloss:0.7369 eloss:1.1775 aloss2:3.2391 exploreP:0.3549
Episode:324 meanR:37.9100 R:32.0000 rate:0.0640 aloss:0.7303 eloss:1.2013 aloss2:3.2941 exploreP:0.3538
Episode:325 meanR:37.9300 R:57.0000 rate:0.1140 aloss:0.7095 eloss:1.1921 aloss2:3.2823 exploreP:0.3518
Episode:326 meanR:37.9600 R:35.0000 rate:0.0700 aloss:0.7233 elo

...

Episode:2742 meanR:219.8000 R:500.0000 rate:1.0000 aloss:0.3147 eloss:1.5235 aloss2:2.7607 exploreP:0.0100
Episode:2743 meanR:220.1800 R:149.0000 rate:0.2980 aloss:0.3126 eloss:1.5066 aloss2:2.8830 exploreP:0.0100
Episode:2744 meanR:220.4800 R:114.0000 rate:0.2280 aloss:0.3176 eloss:1.4873 aloss2:2.6448 exploreP:0.0100
Episode:2745 meanR:218.4200 R:96.0000 rate:0.1920 aloss:0.3209 eloss:1.5083 aloss2:2.6707 exploreP:0.0100
Episode:2746 meanR:222.7100 R:500.0000 rate:1.0000 aloss:0.3124 eloss:1.4754 aloss2:2.8517 exploreP:0.0100
Episode:2747 meanR:219.6300 R:64.0000 rate:0.1280 aloss:0.3129 eloss:1.4357 aloss2:2.9823 exploreP:0.0100
Episode:2748 meanR:222.1500 R:500.0000 rate:1.0000 aloss:0.3133 eloss:1.4301 aloss2:2.9734 exploreP:0.0100
Episode:2749 meanR:222.3800 R:107.0000 rate:0.2140 aloss:0.3116 eloss:1.4167 aloss2:2.9173 exploreP:0.0100
Episode:2750 meanR:224.6600 R:332.0000 rate:0.6640 aloss:0.3125 eloss:1.4040 aloss2:2.9860 exploreP:0.0100
Episode:2751 meanR:224.8200 R:163.0000 

Episode:2819 meanR:319.8600 R:500.0000 rate:1.0000 aloss:0.3062 eloss:1.1502 aloss2:5.3166 exploreP:0.0100
Episode:2820 meanR:324.2100 R:500.0000 rate:1.0000 aloss:0.3047 eloss:1.1556 aloss2:5.7408 exploreP:0.0100
Episode:2821 meanR:325.1300 R:293.0000 rate:0.5860 aloss:0.3019 eloss:1.1393 aloss2:5.3214 exploreP:0.0100
Episode:2822 meanR:325.1300 R:500.0000 rate:1.0000 aloss:0.3026 eloss:1.1405 aloss2:5.8315 exploreP:0.0100
Episode:2823 meanR:321.1300 R:100.0000 rate:0.2000 aloss:0.2992 eloss:1.1464 aloss2:6.1620 exploreP:0.0100
Episode:2824 meanR:320.5900 R:89.0000 rate:0.1780 aloss:0.3053 eloss:1.1116 aloss2:5.5334 exploreP:0.0100
Episode:2825 meanR:324.7400 R:500.0000 rate:1.0000 aloss:0.3093 eloss:1.1056 aloss2:5.7453 exploreP:0.0100
Episode:2826 meanR:324.7400 R:500.0000 rate:1.0000 aloss:0.2990 eloss:1.1316 aloss2:6.6151 exploreP:0.0100
Episode:2827 meanR:324.7400 R:500.0000 rate:1.0000 aloss:0.3035 eloss:1.1128 aloss2:6.5274 exploreP:0.0100
Episode:2828 meanR:322.0300 R:229.0000

Episode:2896 meanR:371.1900 R:500.0000 rate:1.0000 aloss:0.2376 eloss:0.9036 aloss2:13.5666 exploreP:0.0100
Episode:2897 meanR:371.1900 R:500.0000 rate:1.0000 aloss:0.2266 eloss:0.8851 aloss2:12.5021 exploreP:0.0100
Episode:2898 meanR:375.1600 R:500.0000 rate:1.0000 aloss:0.2197 eloss:0.9197 aloss2:13.6081 exploreP:0.0100
Episode:2899 meanR:375.1600 R:500.0000 rate:1.0000 aloss:0.2273 eloss:0.8825 aloss2:12.1959 exploreP:0.0100
Episode:2900 meanR:371.7000 R:154.0000 rate:0.3080 aloss:0.2174 eloss:0.8872 aloss2:12.1667 exploreP:0.0100
Episode:2901 meanR:375.6800 R:500.0000 rate:1.0000 aloss:0.2294 eloss:0.8955 aloss2:12.0502 exploreP:0.0100
Episode:2902 meanR:379.5000 R:500.0000 rate:1.0000 aloss:0.2309 eloss:0.8681 aloss2:11.4600 exploreP:0.0100
Episode:2903 meanR:379.1800 R:136.0000 rate:0.2720 aloss:0.2291 eloss:0.8592 aloss2:11.1851 exploreP:0.0100
Episode:2904 meanR:379.7700 R:152.0000 rate:0.3040 aloss:0.2216 eloss:0.9142 aloss2:12.1535 exploreP:0.0100
Episode:2905 meanR:379.7700 

Episode:2972 meanR:357.9900 R:500.0000 rate:1.0000 aloss:0.1934 eloss:0.7829 aloss2:29.4385 exploreP:0.0100
Episode:2973 meanR:357.9900 R:500.0000 rate:1.0000 aloss:0.2015 eloss:0.7481 aloss2:24.3329 exploreP:0.0100
Episode:2974 meanR:358.5900 R:147.0000 rate:0.2940 aloss:0.1987 eloss:0.7530 aloss2:24.5641 exploreP:0.0100
Episode:2975 meanR:358.5900 R:500.0000 rate:1.0000 aloss:0.1961 eloss:0.7435 aloss2:24.5032 exploreP:0.0100
Episode:2976 meanR:358.5900 R:500.0000 rate:1.0000 aloss:0.1944 eloss:0.7496 aloss2:23.6673 exploreP:0.0100
Episode:2977 meanR:358.2900 R:100.0000 rate:0.2000 aloss:0.1993 eloss:0.7473 aloss2:30.0474 exploreP:0.0100
Episode:2978 meanR:358.3800 R:93.0000 rate:0.1860 aloss:0.2015 eloss:0.7716 aloss2:39.2709 exploreP:0.0100
Episode:2979 meanR:358.3800 R:500.0000 rate:1.0000 aloss:0.1991 eloss:0.7205 aloss2:20.7480 exploreP:0.0100
Episode:2980 meanR:358.3800 R:500.0000 rate:1.0000 aloss:0.1972 eloss:0.7467 aloss2:24.3377 exploreP:0.0100
Episode:2981 meanR:358.3800 R

Episode:3048 meanR:398.3100 R:500.0000 rate:1.0000 aloss:0.1713 eloss:0.7036 aloss2:39.3184 exploreP:0.0100
Episode:3049 meanR:398.3100 R:500.0000 rate:1.0000 aloss:0.1721 eloss:0.7031 aloss2:40.3699 exploreP:0.0100
Episode:3050 meanR:398.3100 R:500.0000 rate:1.0000 aloss:0.1692 eloss:0.7024 aloss2:41.0219 exploreP:0.0100
Episode:3051 meanR:398.3100 R:500.0000 rate:1.0000 aloss:0.1713 eloss:0.7015 aloss2:46.6124 exploreP:0.0100
Episode:3052 meanR:398.3100 R:500.0000 rate:1.0000 aloss:0.1700 eloss:0.7011 aloss2:43.8060 exploreP:0.0100
Episode:3053 meanR:398.3100 R:500.0000 rate:1.0000 aloss:0.1693 eloss:0.7008 aloss2:44.7025 exploreP:0.0100
Episode:3054 meanR:402.1500 R:500.0000 rate:1.0000 aloss:0.1693 eloss:0.7006 aloss2:46.0560 exploreP:0.0100
Episode:3055 meanR:405.8000 R:500.0000 rate:1.0000 aloss:0.1707 eloss:0.7004 aloss2:47.8433 exploreP:0.0100
Episode:3056 meanR:405.8000 R:500.0000 rate:1.0000 aloss:0.1707 eloss:0.6997 aloss2:52.4043 exploreP:0.0100
Episode:3057 meanR:409.7400 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
